<a href="https://colab.research.google.com/github/MohamedAbuAmira/ML-DL-Examples/blob/master/Text_generation_with_an_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


from https://www.tensorflow.org/tutorials/text/text_generation



This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's The Unreasonable Effectiveness of Recurrent Neural Networks. Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

# Setup

In [3]:
import tensorflow as tf

import numpy as np
import os
import time

In [4]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt',
                                       'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


## Read the data

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [8]:
print(vocab)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# Process the text

## Vectorize the text

* Before training, we need to map strings to a numerical representation. 
* **Create two lookup tables**: 
 * one mapping characters to numbers, 
 * and another for numbers to characters.

In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

**Now we have an integer representation for each character. Notice that we mapped the character as indexes from 0 to len(unique).**

In [10]:
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [11]:
print(idx2char)

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z']


In [12]:
print(len(text_as_int))

1115394


In [13]:
print(text_as_int[:250])

[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59  1 39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39
 58 46 43 56  1 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47
 57 46 12  0  0 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53
 50 60 43 42  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47
 56 57 58  6  1 63 53 59  1 49 52 53 61  1 15 39 47 59 57  1 25 39 56 41
 47 59 57  1 47 57  1 41 46 47 43 44  1 43 52 43 51 63  1 58 53  1 58 46
 43  1 54 43 53 54 50 43  8  0]


In [14]:
print(idx2char[text_as_int[:250]])

['F' 'i' 'r' 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'B' 'e' 'f'
 'o' 'r' 'e' ' ' 'w' 'e' ' ' 'p' 'r' 'o' 'c' 'e' 'e' 'd' ' ' 'a' 'n' 'y'
 ' ' 'f' 'u' 'r' 't' 'h' 'e' 'r' ',' ' ' 'h' 'e' 'a' 'r' ' ' 'm' 'e' ' '
 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'A' 'l' 'l' ':' '\n' 'S' 'p' 'e' 'a'
 'k' ',' ' ' 's' 'p' 'e' 'a' 'k' '.' '\n' '\n' 'F' 'i' 'r' 's' 't' ' ' 'C'
 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'Y' 'o' 'u' ' ' 'a' 'r' 'e' ' ' 'a' 'l'
 'l' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd' ' ' 'r' 'a' 't' 'h' 'e' 'r' ' '
 't' 'o' ' ' 'd' 'i' 'e' ' ' 't' 'h' 'a' 'n' ' ' 't' 'o' ' ' 'f' 'a' 'm'
 'i' 's' 'h' '?' '\n' '\n' 'A' 'l' 'l' ':' '\n' 'R' 'e' 's' 'o' 'l' 'v'
 'e' 'd' '.' ' ' 'r' 'e' 's' 'o' 'l' 'v' 'e' 'd' '.' '\n' '\n' 'F' 'i' 'r'
 's' 't' ' ' 'C' 'i' 't' 'i' 'z' 'e' 'n' ':' '\n' 'F' 'i' 'r' 's' 't' ','
 ' ' 'y' 'o' 'u' ' ' 'k' 'n' 'o' 'w' ' ' 'C' 'a' 'i' 'u' 's' ' ' 'M' 'a'
 'r' 'c' 'i' 'u' 's' ' ' 'i' 's' ' ' 'c' 'h' 'i' 'e' 'f' ' ' 'e' 'n' 'e'
 'm' 'y' ' ' 't' 'o' ' ' 't' 'h' 'e' ' ' 'p' '

In [15]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [16]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


## The prediction task

* Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. 
* The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since *** RNNs maintain an internal state that depends on the previously seen elements**, *given all the characters computed until this moment*, **what is the next character?**

## Create training examples and targets

* Next divide the text into example sequences. Each input sequence will contain **seq_length** characters from the text.

* For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

* So break the text into chunks of `seq_length+1`. 
* For example, say **seq_length** is 4 and our text is "**Hello**". The input sequence would be "Hell", and the target sequence "ello".



To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [17]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [18]:
print("examples_per_epoch: ", examples_per_epoch)

examples_per_epoch:  11043


The `batch` method lets us easily convert these individual characters to sequences of the desired size.



In [19]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  sample_=idx2char[item.numpy()]
  print(repr(''.join(sample_)))
  

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
print(len(sample_))

101


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [21]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:



In [22]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


* Each index of these vectors are processed as one time step. 
* For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. 
* At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.

In [23]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
  print("Step {:4d}".format(i))
  print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print("  expected output: {} ({:s})\n".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')

Step    1
  input: 47 ('i')
  expected output: 56 ('r')

Step    2
  input: 56 ('r')
  expected output: 57 ('s')

Step    3
  input: 57 ('s')
  expected output: 58 ('t')

Step    4
  input: 58 ('t')
  expected output: 1 (' ')



## Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

# Build The Model

* Use **tf.keras.Sequential** to define the model. For this simple example three layers are used to define our model:

 1. **tf.keras.layers.Embedding**: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
 2. **tf.keras.layers.GRU**: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
 3. **tf.keras.layers.Dense**: The output layer, with` vocab_size` outputs.

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [27]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

# Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [29]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


* To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. 
* This distribution is defined by the logits over the character vocabulary.

Try it for the first example in the batch:



In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [31]:
sampled_indices

array([38, 44, 22, 41, 54, 61, 34,  3, 23, 21, 51,  7, 11,  3,  5, 10, 29,
        2, 28, 38, 61, 25, 38,  3, 47,  4, 51, 39, 54, 12, 21, 44,  7, 14,
       13, 14, 40, 55, 61, 59, 36, 24, 11, 59, 51, 52, 31, 64, 16, 39, 17,
       34, 58, 12, 63, 43, 16, 23, 62, 29, 43, 51,  0, 23, 37, 19, 11, 59,
       62, 27,  0, 46, 34, 56, 10,  6, 53, 53, 56, 39, 44, 11, 42, 54, 19,
       38, 52, 45, 49,  0, 28, 37, 36, 49, 14,  0, 20, 56, 44, 24])

Decode these to see the text predicted by this untrained model:



In [32]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ', be bold.\n\nNORTHUMBERLAND:\nThen thus: I have from Port le Blanc, a bay\nIn Brittany, received intell'

Next Char Predictions: 
 "ZfJcpwV$KIm-;$':Q!PZwMZ$i&map?If-BABbqwuXL;umnSzDaEVt?yeDKxQem\nKYG;uxO\nhVr:,ooraf;dpGZngk\nPYXkB\nHrfL"


# Train the model

* At this point the problem can be treated as a standard classification problem. 
* Given the previous RNN state, and the input this time step, predict the class of the next character.

## Attach an optimizer, and a loss function

The standard **tf.keras.losses.sparse_categorical_crossentropy** loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.



In [33]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [34]:

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.174115


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [35]:
model.compile(optimizer='adam', loss=loss)


## Configure checkpoints

Use a **tf.keras.callbacks.ModelCheckpoint** to ensure that checkpoints are saved during training:

In [37]:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'



# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [38]:
EPOCHS=10


In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
172/172 [==============================] - 21s 125ms/step - loss: 2.6624
Epoch 2/10
172/172 [==============================] - 21s 125ms/step - loss: 1.9463
Epoch 3/10
172/172 [==============================] - 22s 126ms/step - loss: 1.6809
Epoch 4/10
172/172 [==============================] - 22s 125ms/step - loss: 1.5352
Epoch 5/10
172/172 [==============================] - 22s 125ms/step - loss: 1.4489
Epoch 6/10
172/172 [==============================] - 22s 125ms/step - loss: 1.3897
Epoch 7/10
172/172 [==============================] - 22s 126ms/step - loss: 1.3449
Epoch 8/10
172/172 [==============================] - 22s 126ms/step - loss: 1.3049
Epoch 9/10
172/172 [==============================] - 22s 126ms/step - loss: 1.2691
Epoch 10/10
172/172 [==============================] - 21s 125ms/step - loss: 1.2366


# Generate text

## Restore the latest checkpoint

* To keep this prediction step simple, use a batch size of 1.

* Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

* To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [42]:
weights=tf.train.latest_checkpoint(checkpoint_dir)
weights


'./training_checkpoints/ckpt_10'

In [43]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(weights)

model.build(tf.TensorShape([1, None]))

In [44]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_2 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## The prediction loop


The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

![picture](https://www.tensorflow.org/tutorials/text/images/text_generation_sampling.png)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [45]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [46]:
print(generate_text(model, start_string=u"ROMEO: "))


ROMEO: I am thence.

GREMIO:
Believe these news?

GONZALO:
Another blace,
Speak tekes with denaltions come,
But confesses age and melant grave and kill on it arm'd!
And kake me fault,
Infurn his majesty fight, Angelo's ancient.

KATHARINA:
Heaving for my mouth, a coil, and to the falter!

BATHAS flain from the point o' the feen or thou.

KING EDWARD IV:
Urght our daughter and they govern!

EDWARD:
O Herting, therefore, well; good new men behind--
Which, to pieces o' the offence
But Romem here in a dream.

KING RICHARD III:
An all my son
While his my prayers for this porient dark,
Though he hath come unto her eyeche fault?
Of sal an hour still submers may dare to fight: finding cold.

KING RICHARD III:
Romeo, you should remain.

SICINIUS:
Well, lear thy father.

PRIVERLE:
He that were not spifit; with use
it issue; do, go: but how he were,
A prayer, such when the other stand ne--this reason, which, for him; he lives all.
I am not winter time and known sink with rage.

GLOUCESTER:
The tr

The easiest thing you can do to improve the results it to train it for longer (try EPOCHS=30).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

# Advanced: Customized Training

* The above training procedure is simple, but does not give you much control.

* So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. 
* This gives a starting point if, for example, to implement curriculum learning to help stabilize the model's open-loop output.

We will use **tf.GradientTape** to track the gradients. You can learn more about this approach by reading the eager execution guide.

**The procedure works as follows:**

* First, reset the RNN state. We do this by calling the **tf.keras.Model.reset_states** method.

* Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.

* Open a **tf.GradientTape**, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.

In [47]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [48]:
optimizer = tf.keras.optimizers.Adam()


In [49]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [50]:
# Training step
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.173279762268066
Epoch 1 Batch 100 Loss 2.3352110385894775
Epoch 1 Loss 2.1982
Time taken for 1 epoch 23.91321039199829 sec

Epoch 2 Batch 0 Loss 2.1554675102233887
Epoch 2 Batch 100 Loss 1.9612998962402344
Epoch 2 Loss 1.7754
Time taken for 1 epoch 22.44538688659668 sec

Epoch 3 Batch 0 Loss 1.790032148361206
Epoch 3 Batch 100 Loss 1.6777257919311523
Epoch 3 Loss 1.6273
Time taken for 1 epoch 22.210379600524902 sec

Epoch 4 Batch 0 Loss 1.5384907722473145
Epoch 4 Batch 100 Loss 1.5245368480682373
Epoch 4 Loss 1.5596
Time taken for 1 epoch 22.173303604125977 sec

Epoch 5 Batch 0 Loss 1.4695391654968262
Epoch 5 Batch 100 Loss 1.4586117267608643
Epoch 5 Loss 1.4569
Time taken for 1 epoch 22.322993516921997 sec

Epoch 6 Batch 0 Loss 1.3666186332702637
Epoch 6 Batch 100 Loss 1.4104125499725342
Epoch 6 Loss 1.3833
Time taken for 1 epoch 22.292937994003296 sec

Epoch 7 Batch 0 Loss 1.3247041702270508
Epoch 7 Batch 100 Loss 1.363092303276062
Epoch 7 Loss 1.3137
Time take

In [51]:
model.save_weights(checkpoint_prefix.format(epoch=epoch))


## Predict

In [52]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_29'

In [53]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [54]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: what then?

Messenger:
Well, let me know my heart.

DERBY:
Bear her that he will sovere man that fair speed;
Earl of Northumberland, this news with me
How now that God he sled to be
Myself am high with gelivy to your pilgrimage; for then I'll make a life
With you; there camest things venture,
The trade and season, no other-in-law.

CAPULET:
Here comest thou never come.

BUCKINGHAM:
Why, that shall die?

DUKE OF YORK:
To murder me?

Second Lady consumest incase to sea and sense
That west thou Rocaverties:
Farewell; thou hast lived, or not upon him
stuffice of our sheep-sheating suns.

LADY ANNE:
Would they are deceived; and how his face is this;
that I have heard
out, hadst thou not stay to hear it.

HENRY BOLINGBROKE:
Good you would please you, sir: I'll not forsook;
And speak good until with themselves; mut it fellew her son:
Then, dame you now, sir? the pleasure of the capak of this business?
If I perceive thou hadst foreto-morrow.

QUEEN MARGARET:
Walt revengelo, to said thei